# Proyecto Integrador III

## Diego Lopez Castan

In [3]:
# Instalo librerias necesarias
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [2]:
# Creo los datasets desde los archivos
categories = pd.read_csv("categories.csv")
cities = pd.read_csv("cities.csv")
countries = pd.read_csv("countries.csv")
customers = pd.read_csv("customers.csv")
employees = pd.read_csv("employees.csv")
products = pd.read_csv("products.csv")
sales = pd.read_csv("sales.csv")

In [3]:
# Verifico los registros de cada dataset
print("Cantidad de registros del dataframe categories:", len(categories))
print("Cantidad de registros del dataframe cities:", len(cities))
print("Cantidad de registros del dataframe countries:", len(countries))
print("Cantidad de registros del dataframe customers:", len(customers))
print("Cantidad de registros del dataframe employees:", len(employees))
print("Cantidad de registros del dataframe products:", len(products))
print("Cantidad de registros del dataframe sales:", len(sales))

Cantidad de registros del dataframe categories: 11
Cantidad de registros del dataframe cities: 96
Cantidad de registros del dataframe countries: 206
Cantidad de registros del dataframe customers: 98759
Cantidad de registros del dataframe employees: 23
Cantidad de registros del dataframe products: 452
Cantidad de registros del dataframe sales: 6758125


In [4]:
# Hago una copia del dataframe sales para procesarlo
sales_df = sales.copy()

In [5]:
# Uno la tabla sales con products usando ProductID (o nombre similar)
sales_df = sales.merge(products[['ProductID', 'Price']], on='ProductID', how='left')

In [6]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780


In [7]:
# Creo el campo TotalPriceCalculated
sales_df['TotalPriceCalculated'] = (sales_df['Quantity'] * sales_df['Price']) * (1 - sales_df['Discount'])

In [8]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200


In [9]:
# Detecto los outliers
# Calculo los cuartiles e IQR

Q1 = sales_df['TotalPriceCalculated'].quantile(0.25)
Q3 = sales_df['TotalPriceCalculated'].quantile(0.75)
IQR = Q3 - Q1

In [10]:
# Calculo los límites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [11]:
# Creo el campo IsOutlier para identificar si el dato TotalPriceCalculated es outlier

sales_df['IsOutlier'] = ((sales_df['TotalPriceCalculated'] < lower_bound) |
                      (sales_df['TotalPriceCalculated'] > upper_bound)).astype(int)

In [12]:
# Verifico los outliers
num_outliers = sales_df['IsOutlier'].sum()
print(f"Se detectaron {num_outliers} outliers.")

Se detectaron 48217 outliers.


In [13]:
# Crea una nueva columna que contenga únicamente la hora de la venta.
# Primero cambio el tipo de dato del campo Salesdate

sales_df['SalesDate'] = pd.to_datetime(sales_df['SalesDate'])

In [14]:
sales_df['Hour'] = sales_df['SalesDate'].dt.hour

In [15]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated,IsOutlier,Hour
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,309.63590,0,7.0
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,437.82200,0,16.0
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,1896.44160,0,19.0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,1236.01384,0,14.0
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,719.80200,0,15.0


In [16]:
sales_hour = sales_df.groupby('Hour')['TotalPriceCalculated'].sum().reset_index()

In [17]:
pd.set_option('display.float_format', '{:,.2f}'.format) # Con esta opcion hago que el campo TotalPriceCalculated no aparezca con notacion científica
sales_hour

,Hour,TotalPriceCalculated
0,0.00,"178,313,404.12"
1,1.00,"177,823,245.94"
2,2.00,"178,420,846.95"
3,3.00,"177,443,825.95"
4,4.00,"177,932,710.89"
5,5.00,"177,506,190.92"
6,6.00,"178,381,199.01"
7,7.00,"177,817,815.98"
8,8.00,"177,882,244.32"
9,9.00,"178,166,580.43"


In [18]:
# Busco el mayor valor del campo TotalPriceCalculated
max_hour = sales_hour[sales_hour['TotalPriceCalculated'] == sales_hour['TotalPriceCalculated'].max()]

In [19]:
max_hour

,Hour,TotalPriceCalculated
16,16.00,"179,014,421.24"


In [20]:
# Obtengo el número del día (0=lunes, 6=domingo)
sales_df['Weekday'] = sales_df['SalesDate'].dt.weekday

In [21]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated,IsOutlier,Hour,Weekday
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64,0,7.00,0.00
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82,0,16.00,4.00
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44",0,19.00,3.00
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01",0,14.00,5.00
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80,0,15.00,0.00


In [22]:
# Clasifico si es dia de semana o fin de semana
sales_df['DayType'] = sales_df['Weekday'].apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

In [23]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated,IsOutlier,Hour,Weekday,DayType
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64,0,7.00,0.00,Weekday
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82,0,16.00,4.00,Weekday
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44",0,19.00,3.00,Weekday
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01",0,14.00,5.00,Weekend
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80,0,15.00,0.00,Weekday


In [24]:
# Calculo las ventas por tipo de día
sales_daytype = sales_df.groupby('DayType')['TotalPriceCalculated'].sum().reset_index()

In [25]:
sales_daytype

,DayType,TotalPriceCalculated
0,Weekday,"3,080,351,837.91"
1,Weekend,"1,235,915,841.01"


In [26]:
# Cambio el tipo del campo de la tabla employee

employees['BirthDate'] = pd.to_datetime(employees['BirthDate'], errors='coerce')
employees['HireDate'] = pd.to_datetime(employees['HireDate'], errors='coerce')

In [27]:
sales_df = sales_df.merge(
    employees[['EmployeeID', 'BirthDate', 'Gender', 'CityID', 'HireDate']],
    left_on='SalesPersonID',
    right_on='EmployeeID',
    how='left'
)

In [28]:
sales_df

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,TotalPriceCalculated,IsOutlier,Hour,Weekday,DayType,EmployeeID,BirthDate,Gender,CityID,HireDate
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,309.64,0,7.00,0.00,Weekday,6,1987-01-13,M,65,2013-06-22 13:20:18.080
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,437.82,0,16.00,4.00,Weekday,16,1951-07-07,M,28,2017-02-10 11:21:26.650
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,"1,896.44",0,19.00,3.00,Weekday,13,1963-04-18,M,68,2011-12-12 10:43:52.940
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,"1,236.01",0,14.00,5.00,Weekend,8,1956-12-13,M,18,2014-10-14 23:12:53.420
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,719.80,0,15.00,0.00,Weekday,10,1963-12-30,M,9,2012-07-23 15:02:12.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758120,6758121,10,50938,300,13,0.00,0.00,2018-04-20 22:39:03.150,AHVBHVO7632V5YYD2VJT,69.57,904.40,0,22.00,4.00,Weekday,10,1963-12-30,M,9,2012-07-23 15:02:12.640
6758121,6758122,18,85668,452,22,0.10,0.00,2018-01-13 15:08:18.700,M4IA0FS13J9F0GJD4BX8,28.57,565.59,0,15.00,5.00,Weekend,18,1964-03-09,M,58,2010-02-22 21:55:48.620
6758122,6758123,21,68409,405,18,0.00,0.00,2018-01-18 12:05:16.950,7VZJ77UGBSX2SNCC1GFW,0.43,7.70,0,12.00,3.00,Weekday,21,1966-04-25,M,65,2016-01-24 02:13:24.340
6758123,6758124,17,90315,104,23,0.00,0.00,2018-02-26 08:13:37.850,3JTF5IF0YJLLXF9IRS8X,98.25,"2,259.68",1,8.00,0.00,Weekday,17,1989-08-29,M,80,2010-07-11 09:23:03.490


In [29]:
# Calculo la edad al momento de su contratación

sales_df['Age_hire'] = (sales_df['HireDate'] - sales_df['BirthDate']).dt.days // 365

In [30]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,IsOutlier,Hour,Weekday,DayType,EmployeeID,BirthDate,Gender,CityID,HireDate,Age_hire
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,0,7.00,0.00,Weekday,6,1987-01-13,M,65,2013-06-22 13:20:18.080,26
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,0,16.00,4.00,Weekday,16,1951-07-07,M,28,2017-02-10 11:21:26.650,65
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,0,19.00,3.00,Weekday,13,1963-04-18,M,68,2011-12-12 10:43:52.940,48
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,0,14.00,5.00,Weekend,8,1956-12-13,M,18,2014-10-14 23:12:53.420,57
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,0,15.00,0.00,Weekday,10,1963-12-30,M,9,2012-07-23 15:02:12.640,48


In [31]:
# Calculo años de experiencia del empleado

sales_df['Years_Experience'] = (sales_df['SalesDate'] - sales_df['HireDate']).dt.days // 365

In [32]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,Hour,Weekday,DayType,EmployeeID,BirthDate,Gender,CityID,HireDate,Age_hire,Years_Experience
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,7.00,0.00,Weekday,6,1987-01-13,M,65,2013-06-22 13:20:18.080,26,4.00
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,16.00,4.00,Weekday,16,1951-07-07,M,28,2017-02-10 11:21:26.650,65,0.00
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,19.00,3.00,Weekday,13,1963-04-18,M,68,2011-12-12 10:43:52.940,48,6.00
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,14.00,5.00,Weekend,8,1956-12-13,M,18,2014-10-14 23:12:53.420,57,3.00
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,15.00,0.00,Weekday,10,1963-12-30,M,9,2012-07-23 15:02:12.640,48,5.00


In [33]:
# Uno mi dataframe con el dataframe de productos

sales_df = sales_df.merge(
    products[['ProductID', 'CategoryID', 'Class', 'Resistant', 'IsAllergic', 'VitalityDays']],
    on='ProductID',
    how='left'
)


In [34]:
# Uno mi dataframe con el dataframe de categorias

sales_df = sales_df.merge(
    categories[['CategoryID', 'CategoryName']],
    on='CategoryID',
    how='left'
)


In [35]:
# Uno mi dataframe con el dataframe de clientes

sales_df = sales_df.merge(
    customers[['CustomerID', 'CityID']].rename(columns={'CityID': 'CityID_Customer'}),
    on='CustomerID',
    how='left'
)


In [36]:
# Label Encoding para variables con pocos valores ordenables o binarias
label_cols = ['DayType', 'Gender', 'Class']
label_encoders = {}

for col in label_cols:
    le = LabelEncoder()
    sales_df[col] = le.fit_transform(sales_df[col])
    label_encoders[col] = le

# One-Hot Encoding para variables categóricas nominales (no ordenables)
one_hot_cols = ['Resistant', 'IsAllergic', 'CategoryName']
df = pd.get_dummies(sales_df, columns=one_hot_cols, drop_first=True)


In [37]:
sales_df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,HireDate,Age_hire,Years_Experience,CategoryID,Class,Resistant,IsAllergic,VitalityDays,CategoryName,CityID_Customer
0,1,6,27039,381,7,0.00,0.00,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.23,...,2013-06-22 13:20:18.080,26,4.00,1.00,0,Unknown,Unknown,41.00,Confections,54
1,2,16,25011,61,7,0.00,0.00,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.55,...,2017-02-10 11:21:26.650,65,0.00,8.00,1,Durable,FALSE,90.00,Grain,71
2,3,13,94024,23,24,0.00,0.00,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.02,...,2011-12-12 10:43:52.940,48,6.00,11.00,2,Durable,TRUE,0.00,Produce,2
3,4,8,73966,176,19,0.20,0.00,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.32,...,2014-10-14 23:12:53.420,57,3.00,6.00,0,Durable,TRUE,90.00,Seafood,45
4,5,10,32653,310,9,0.00,0.00,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.98,...,2012-07-23 15:02:12.640,48,5.00,9.00,1,Weak,FALSE,0.00,Poultry,82


In [38]:
# Grabo los datos de mi dataframe a un archivo csv

sales_df.to_csv('sales_df.csv', index=False)


In [4]:
# Creo dataframe df con los datos procesados

df = pd.read_csv('sales_df.csv')

In [5]:
df.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Price,...,HireDate,Age_hire,Years_Experience,CategoryID,Class,Resistant,IsAllergic,VitalityDays,CategoryName,CityID_Customer
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,...,2013-06-22 13:20:18.080,26,4.0,1.0,0,Unknown,Unknown,41.0,Confections,54
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,...,2017-02-10 11:21:26.650,65,0.0,8.0,1,Durable,FALSE,90.0,Grain,71
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,...,2011-12-12 10:43:52.940,48,6.0,11.0,2,Durable,TRUE,0.0,Produce,2
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,...,2014-10-14 23:12:53.420,57,3.0,6.0,0,Durable,TRUE,90.0,Seafood,45
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,...,2012-07-23 15:02:12.640,48,5.0,9.0,1,Weak,FALSE,0.0,Poultry,82
